In [1]:
#dependencies
import pandas as pd
import requests
import gmaps
from config import gkey

gmaps.configure(api_key=gkey)

In [2]:
#import weather_database.csv
city_data_df = pd.read_csv("WeatherPy_Database2.csv")
city_data_df.head()

,City ID,City,Country,Date,Lat,Lng,Min Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Ushuaia,AR,2022-01-29 20:40:06,-54.8000,-68.3000,46.06,46.06,93,75,25.32,moderate rain
1,1,Hilo,US,2022-01-29 20:37:14,19.7297,-155.0900,73.40,78.53,61,100,3.44,overcast clouds
2,2,Te Anau,NZ,2022-01-29 20:41:05,-45.4167,167.7167,57.67,57.67,88,100,1.41,light rain
3,3,Dunmore East,IE,2022-01-29 20:41:06,52.1514,-6.9861,40.62,44.69,77,20,1.14,few clouds
4,4,Puerto Ayora,EC,2022-01-29 20:37:34,-0.7393,-90.3518,80.56,80.56,71,28,9.35,scattered clouds


In [3]:
#prompt the user to enter desired temperatures
max_temp = float(input("What is the maximum temperature you would like on your trip? Enter the number only. "))
min_temp = float(input("What is the minimum temperature you would like on your trip? Enter the number only. "))

What is the maximum temperature you would like on your trip? Enter the number only. 95
What is the minimum temperature you would like on your trip? Enter the number only. 60


In [4]:
# find the city temperatures that align with the user's request
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                      (city_data_df["Min Temp"] >= min_temp)]

preferred_cities_df

,City ID,City,Country,Date,Lat,Lng,Min Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Hilo,US,2022-01-29 20:37:14,19.7297,-155.0900,73.40,78.53,61,100,3.44,overcast clouds
4,4,Puerto Ayora,EC,2022-01-29 20:37:34,-0.7393,-90.3518,80.56,80.56,71,28,9.35,scattered clouds
6,6,Lompoc,US,2022-01-29 20:41:07,34.6391,-120.4579,60.69,69.19,40,0,0.00,clear sky
7,7,Kapaa,US,2022-01-29 20:41:07,22.0752,-159.3190,77.32,78.78,72,0,3.00,clear sky
9,9,Arraial Do Cabo,BR,2022-01-29 20:36:59,-22.9661,-42.0278,76.86,76.96,94,75,11.50,broken clouds
...,...,...,...,...,...,...,...,...,...,...,...,...
663,663,Giporlos,PH,2022-01-29 21:00:21,11.1200,125.4494,77.09,77.09,90,46,0.83,scattered clouds
665,665,Port Moresby,PG,2022-01-29 21:00:22,-9.4431,147.1797,77.07,77.07,82,100,7.11,heavy intensity rain
667,667,Burnie,AU,2022-01-29 21:00:24,-41.0667,145.9167,60.94,60.94,93,73,7.45,broken clouds
669,669,Siguiri,GN,2022-01-29 21:00:24,11.6667,-9.5000,81.32,81.32,16,3,3.76,clear sky


In [5]:
#find empty rows
preferred_cities_df.isnull()

,City ID,City,Country,Date,Lat,Lng,Min Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
663,False,False,False,False,False,False,False,False,False,False,False,False
665,False,False,False,False,False,False,False,False,False,False,False,False
667,False,False,False,False,False,False,False,False,False,False,False,False
669,False,False,False,False,False,False,False,False,False,False,False,False


In [6]:
#drop any empty rows
clean_preferred_cities_df = preferred_cities_df.dropna()

In [7]:
#create hotel DF
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Hilo,US,78.53,overcast clouds,19.7297,-155.0900,
4,Puerto Ayora,EC,80.56,scattered clouds,-0.7393,-90.3518,
6,Lompoc,US,69.19,clear sky,34.6391,-120.4579,
7,Kapaa,US,78.78,clear sky,22.0752,-159.3190,
9,Arraial Do Cabo,BR,76.96,broken clouds,-22.9661,-42.0278,
10,Busselton,AU,67.05,clear sky,-33.6500,115.3333,
11,Vaini,TO,81.46,overcast clouds,-21.2000,-175.2000,
15,Avarua,CK,82.45,few clouds,-21.2078,-159.7750,
16,Port Elizabeth,ZA,68.29,overcast clouds,-33.9180,25.5701,
18,Butaritari,KI,82.11,light rain,3.0707,172.7902,


In [8]:
#search for hotels within 5000 meters
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey,
    "location": ""}

#iterate through hotel DF

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    #set up the base URL for the Google Direction API to get JSON data
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #retrieve JSON data from the search
    hotels = requests.get(url, params=params).json()

    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel cannot be found...skipping.")
    continue 

Hotel cannot be found...skipping.
Hotel cannot be found...skipping.
Hotel cannot be found...skipping.
Hotel cannot be found...skipping.
Hotel cannot be found...skipping.
Hotel cannot be found...skipping.
Hotel cannot be found...skipping.
Hotel cannot be found...skipping.
Hotel cannot be found...skipping.
Hotel cannot be found...skipping.
Hotel cannot be found...skipping.
Hotel cannot be found...skipping.
Hotel cannot be found...skipping.
Hotel cannot be found...skipping.
Hotel cannot be found...skipping.
Hotel cannot be found...skipping.
Hotel cannot be found...skipping.
Hotel cannot be found...skipping.
Hotel cannot be found...skipping.
Hotel cannot be found...skipping.
Hotel cannot be found...skipping.
Hotel cannot be found...skipping.


In [14]:
clean_hotels_df = hotel_df.dropna()

In [16]:
clean_hotels_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Hilo,US,78.53,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
4,Puerto Ayora,EC,80.56,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
6,Lompoc,US,69.19,clear sky,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
7,Kapaa,US,78.78,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
9,Arraial Do Cabo,BR,76.96,broken clouds,-22.9661,-42.0278,Pousada Porto Praia
...,...,...,...,...,...,...,...
663,Giporlos,PH,77.09,scattered clouds,11.1200,125.4494,JR's Pension House
665,Port Moresby,PG,77.07,heavy intensity rain,-9.4431,147.1797,Lamana Hotel (Port Moresby)
667,Burnie,AU,60.94,broken clouds,-41.0667,145.9167,Beachfront Voyager Motor Inn
669,Siguiri,GN,81.32,clear sky,11.6667,-9.5000,


In [17]:
vacation_search_output = "WeatherPy_vacation.csv"

clean_hotels_df.to_csv(vacation_search_output, index_label="City ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum 
#temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotels_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
travel_hotels_df=clean_hotels_df
locations = travel_hotels_df[["Lat", "Lng"]]

In [29]:
#add heatmap layer with pop-up markers
locations = travel_hotels_df[["Lat", "Lng"]]
max_temp = travel_hotels_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                max_intensity=100, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

In [30]:
fig

Figure(layout=FigureLayout(height='420px'))